In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets

# Load Data

In [2]:
finData = np.loadtxt('tictac_final.txt')
singData = np.loadtxt('tictac_single.txt')
multData = np.loadtxt('tictac_multi.txt')

xSing = singData[:, :9]
ySing = singData[:, 9:].ravel()
xMult = multData[:, :9]
yMult = multData[:, 9:]

# Train Models

In [3]:
# SVM Classifier
from sklearn import svm
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn import metrics

svmClf = svm.SVC(decision_function_shape='ovo').fit(xSing, ySing)

In [4]:
# KNN Classifier
from sklearn.neighbors import KNeighborsClassifier

knnClf = KNeighborsClassifier(algorithm='auto', n_neighbors=8).fit(xSing, ySing)

In [5]:
# MLP Classifier
from sklearn.neural_network import MLPClassifier

mlpClf = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1, max_iter = 500).fit(xSing, ySing)

In [6]:
# KNN Regressor
from sklearn.neighbors import KNeighborsRegressor

# Fit model
knnReg = KNeighborsRegressor(algorithm='auto', n_neighbors=19, weights='distance').fit(xMult, yMult)

In [7]:
# Linear Regressor
from sklearn.linear_model import LinearRegression

# train 9 models, each regressing on individual output index
lin_models = []
for i in range(9):
    # Create and fit a linear regression model for the i-th output dimension
    model = LinearRegression()
    model.fit(xMult, yMult[:, i])  # Training only on the i-th output dimension
    lin_models.append(model)

def getMove(board):    
        preds = []
        for model in lin_models:
            preds.append(model.predict([board])[0])
        for i in range(9):
            optimal_move = np.argmax(preds)
            if board[optimal_move] == 0:
                return optimal_move
            else:
                preds[optimal_move] = 0

In [8]:
# MLP Regressor
from sklearn.neural_network import MLPRegressor

mlpReg = MLPRegressor(alpha=1e-5, max_iter = 500).fit(xMult, yMult)

# Gameplay

In [9]:
# Check if there is a winner
def checkWinner(board):
    if board[0] == board[4] == board[8] and board[0] != 0: #top left to bottom right
        return True
    elif board[2] == board[4] == board[6] and board[2] != 0: #top right corner to bottom left corner
        return True
    for i in range(3): #straight up and down
        if board[i] == board[i + 3] ==board[i + 6] and board[i] != 0: 
            return True
    i = 0
    while i < 9: #straight across
        if board[i] == board[i + 1] == board[i + 2] and board[i] != 0:
            return True
        i += 3

    return False

# Check if board is full (tie condition)
def checkFull(board):
    for i in board:
        if i == 0:
            return False
    return True

# Print the board
def printBoard(board):
    for i in range(9):
        if board[i] == -1:
            print("O", end=" ")
        elif board[i] == 1:
            print("X", end=" ")
        elif board[i] == 0:
            print(" ", end = " ")
        if (i + 1) % 3 == 0:
            print("\n------\n")
    return
            
# Create a 9-element "board"
board = [0,0,0,0,0,0,0,0,0]

# Define players, user (X = 1) and CPU (O = -1)
players = [1, -1]

# Start the game with random player
import random
current_player = random.choice([1, -1])

# Get mode for ML Prediction
mode = input("Select ML Prediction Model:\n 1. SVM Classifier\n 2. KNN Classifier\n 3. MLP Classifier\n 4. KNN Regressor\n 5. Linear Regression\n 6. MLP Regressor\n")
match mode:
    case "1":
        model = svmClf
        print("Using SVM Classification")
    case "2":
        model = knnClf
        print("Using KNN Classification")
    case "3":
        model = mlpClf
        print("Using MLP Classification")
    case "4":
        model = knnReg
        print("Using KNN Regression")
    case "5":
        print("Using Linear Regression")
    case "6":
        model = mlpReg
        print("Using MLP Regression")

# Game loop
while not checkWinner(board) and not checkFull(board):

    # ML Model's turn
    if current_player == -1:
        print("ML Model's Turn:")
        
        # Classifiers
        if(mode in ["1", "2", "3"]):
            move = int(model.predict([board])[0])
        # KNN Regressor
        elif mode == "4":
            y_pred = model.predict([board]) # retreives prediction, 2 dimensional array with only 1 array inside
            for i in range(9):
                y_pred[0][i] = round(y_pred[0][i])  # round the values of the array to 1 or 0
                if int(y_pred[0][i]) != 0 and board[i] == 0: # if value is 1 and array position is empty, choose this one
                    move = i
                    break
        # Linear Regressor
        elif mode == "5":
            move = getMove(board)
        #MLP Regrerssor
        elif mode == "6":
            # chooses move in order of confidence
            y_pred = model.predict([board]) # retreives prediction, 2 dimensional array with only 1 array inside
            #gives 9 tries, in case bioard is full this will allow to choose last empty spot
            for i in range(9):
                move = np.argmax(y_pred[0]) #index of max confidence prediction
                if board[move] != 0:
                    y_pred[move]=0
                else: break

        
        # Check if move is valid
        if move > 8 or move < 0:
            print("ML Model Returned invalid response: ", move)
            break
        
        # Check if space is empty
        if board[move] != 0:
            index = 0
            for i in board:
                if i == 0:
                    move = index
                index+=1
        
        # Make the move
        board[move] = current_player
        
        #Output prediction (1-indexed for user interpretability)
        print(move + 1)
    
    #User's turn
    else:
        # Get user's move
        move = input("Enter your move (1-9): ")

        # Check if move is valid
        if move not in ["1", "2", "3", "4", "5", "6", "7", "8", "9"]:
            print("Invalid move. Please try again.")
            continue

        # Check if space is empty
        if board[int(move) - 1] != 0:
            print("Space is already taken. Please try again.")
            continue

        # Make the move
        board[int(move) - 1] = current_player


    # Switch players, start with user
    current_player = players[players.index(current_player) ^ 1]
    
    # Print the board
    printBoard(board)

if checkWinner(board):
    if players[players.index(current_player) ^ 1] == 1:
        print("You win!")
    else:
        print("The machine has become self aware :P")
else:
    print("It's a draw!")


KeyboardInterrupt: Interrupted by user